In [1]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

$\textbf{BNF Notation:}$

\
\begin{eqnarray}
    \textit{program}         & ::= & \textbf{ Module( } \textit{ statement } \textbf{ ) }\\
                         &     & \\
    \textit{ statement } & ::= & \textbf{ FunctionDef( } \textit{ statement } \textbf{,} \textit{ argument }\textbf{ ) }\\
                         & |   & \textbf{ Return( } \textit{ expression } \textbf{ )}\\
                         & |   & \textbf{ Assign( } \textit{ expression } \textbf{,} \textit{ expression} \textbf{ )}\\
                         & |   & \textbf{ Import( } \textit{ string } \textbf{ )}\\
                         & |   & \textbf{ Expr( } \textit{ expr } \textbf{ )}\\
                         & |   & \textbf{ Pass }\\
                         &     & \\
    \textit{expression}        & ::= & \textbf{BinOp( } \textit{ expression } \textbf{ , } \textit{ operator } \textbf{ , }\textit{ expr } \textbf{ )} \\
                         & |   & \textbf{Num}\\
                         & |   & \textbf{Name}\\
                         & |   & \textbf{Attribute ( } \textit{ Name } \textbf{ )}\\
                         & |   & \textbf{List ( } \textit{ element } \textbf{ )}\\
                         & |   & \textbf{Call(} \textit{ Attribute } \textbf{ )}\\
                         & |   & \textbf{nparray}\\
                         &     & \\
    \textit{argument}    & ::= & expr\\
                         &     & \\
    \textit{operator}    & ::= & \textbf{Add} \\
                         & |   & \textbf{Sub} \\
                         & |   & \textbf{Mult} \\
                         & |   & \textbf{Div} \\
\end{eqnarray}



# Check That A Module That Represents A Program Within The Embedded Language Conforms To The Syntax Of The Language

In [2]:
# Imports to help analyze the AST
import ast 
import inspect
import astor

# Visiting Nodes Of A Module And Checking If It Conforms With The Defined Syntax
class is_validSyntax(ast.NodeVisitor):
    
    def __init__(self):
        self.env = {}
        self.numpyLang = {'np','array', 'dot', 'matmul', 'linalg', 'multi_dot'}
        self.binOps = {ast.Add, ast.Sub, ast.Mult, ast.Div}
    
    def visit_Module(self,node):
        print(type(node))
        results = [self.visit(s) for s in node.body]
        for s in node.body:
            s.env = self.env
        print("Body of Module:", results)
        return all(results)
    
    def visit_FunctionDef(self, node):
        arguments = []
        for i in node.args.args:
            arguments.append(i.arg)
            self.env[i.arg] = True
#         print(arguments)
        print(type(node),": " + node.name + "(" + ', '.join(arguments) +")")
        results = [self.visit(s) for s in node.body]
        print("Body of FunctionDef:", results)
        return all(results)
    
    def visit_Import(self, node):
        print(type(node), [alt.asname for alt in node.names])
        self.env[node.names[0]] = node.names[0].asname
        return True
#         return self.visit()
    
    def visit_Assign(self, node):
        print(type(node))
        env = self.env
        node.value.env = env
        value = self.visit(node.value)
        env[node.targets[0].id] = value
        return value
    
    def visit_Call(self, node):
        print(type(node))
        if type(node.func) is ast.Attribute:
            return self.visit(node.func) and all([self.visit(arguments) for arguments in node.args])
    
    def visit_List(self, node):
        print(type(node))
        return[self.visit(element) for element in node.elts]
    
    def visit_Expr(self, node):
        print(type(node))
        return self.visit(node.value)
    
    def visit_Attribute(self, node):
        print(type(node), node.attr)
        if node.attr in self.numpyLang:
            return self.visit(node.value)
        else:
            return False
        
    def visit_BinOp(self, node):
        print(type(node), type(node.op))
        node.left.env = self.env
        node.right.env = self.env
        if type(node.op) in self.binOps:
            return self.visit(node.left) and self.visit(node.right)
    
    def visit_Name(self, node):
        print(type(node), node.id)
        if node.id in self.env or node.id in self.numpyLang:
            return True

    def visit_Num(self, node):
        print(type(node), node.n)
        return True
        
    def visit_Return(self, node):
        print(type(node))
        return self.visit(node.value)
    
    def visit_Pass(self, node):
        print(type(node))
        return True

#Decorator for checking if a module is valid within the syntax
def is_valid(dump = True, showRes = True):
    def checkValid(f):
        if showRes:
            print("Testing for valid syntax...")
            print("Checking nodes... \n")
            tree = ast.parse(inspect.getsource(f))
            validLang = is_validSyntax().visit(tree)
            print("")
            if dump:
                print("\n------------------------")
                print("Source Code For This Function: ")
                print(astor.to_source(tree))
                print("AST for this function is:")
                print(ast.dump(tree))
                print("------------------------\n")
            
            if validLang:
                print("The function " + str(f.__name__) + "() has valid language. \n")
            else:
                #Show which parts are invalid?
                print("The function " + str(f.__name__) + "() has invalid language. \n")
            print("End of Check...")
            print("======================================================================= \n")
            return validLang
    return checkValid

#Sample Tests For Checking Validity

@is_valid(dump = False, showRes = False)
def simple_testValid():
    import numpy as np
    x = np.array([[1,2],[2,2]])
    return x


@is_valid(dump = False, showRes = False)
def simple_test_Invalid(x):
    import numpy as np
    x = np.random(1,100)
    return x
    

@is_valid(dump = False, showRes = False)
def dot_With_Input_Valid(y):
    import numpy as np
    x = np.array([[1,2],[2,2]])
    return np.dot(x,y)

# Tests with three different matrix multiplication functions

@is_valid(dump = False, showRes = True)
def dot_Valid():
    import numpy as np
    x = np.array([[1,2],[2,2]])
    y = np.array([[99,3],[3,4]])
    return np.dot(x,y)

@is_valid(dump = False, showRes = True)
def matMul_Valid():
    x = np.array([[1,2],[2,2]])
    y = np.array([[99,3],[3,4]])
    return np.matmul(x,y)

@is_valid(dump = True, showRes = True)
def multiDot_Valid():
    import numpy as np
    x = np.array([[1,2],[2,2]])
    y = np.array([[99 + 5,3],[3,4]])
    return np.linalg.multi_dot(x,y)

Testing for valid syntax...
Checking nodes... 

<class '_ast.Module'>
<class '_ast.FunctionDef'> : dot_Valid()
<class '_ast.Import'> ['np']
<class '_ast.Assign'>
<class '_ast.Call'>
<class '_ast.Attribute'> array
<class '_ast.Name'> np
<class '_ast.List'>
<class '_ast.List'>
<class '_ast.Num'> 1
<class '_ast.Num'> 2
<class '_ast.List'>
<class '_ast.Num'> 2
<class '_ast.Num'> 2
<class '_ast.Assign'>
<class '_ast.Call'>
<class '_ast.Attribute'> array
<class '_ast.Name'> np
<class '_ast.List'>
<class '_ast.List'>
<class '_ast.Num'> 99
<class '_ast.Num'> 3
<class '_ast.List'>
<class '_ast.Num'> 3
<class '_ast.Num'> 4
<class '_ast.Return'>
<class '_ast.Call'>
<class '_ast.Attribute'> dot
<class '_ast.Name'> np
<class '_ast.Name'> x
<class '_ast.Name'> y
Body of FunctionDef: [True, True, True, True]
Body of Module: [True]

The function dot_Valid() has valid language. 

End of Check...

Testing for valid syntax...
Checking nodes... 

<class '_ast.Module'>
<class '_ast.FunctionDef'> : matMul_V

# 3(a) 
The Following Code Will Take A Piece Of Code And Return The Dimensions Of Each NP Array In The Code And Will Estimate The Dimensions Of The Resulting Array From Doing NP.Dot On Two Arrays.

In [3]:
class size_Of_Array(ast.NodeVisitor):
    
    def __init__(self):
        self.env = {}
        self.numpyLang = {'np','array', 'dot', 'matmul', 'linalg', 'multi_dot'}
        self.binOps = {'Add', 'Sub', 'Mult', 'Div'}
        self.mult = {}
    
    def visit_Module(self,node):
        print(type(node))
        results = [self.visit(s) for s in node.body]
        for s in node.body:
            s.env = self.env
        print("Body of Module:", results)
        return self.env, self.mult
    
    def visit_FunctionDef(self, node):
        arguments = []
        for i in node.args.args:
            arguments.append(i.arg)
            self.env[i.arg] = i.arg.n
#         print(arguments)
        print(type(node),": " + node.name + "(" + ', '.join(arguments) +")")
        results = [self.visit(s) for s in node.body]
        print("Body of FunctionDef:", results)
        return self.env, self.mult
    
    def visit_Assign(self, node):
        print(type(node))
        env = self.env
        node.value.env = env
        value = self.visit(node.value)
        if not isinstance(value,list):
            env[node.targets[0].id] = value 
        else:
            env[node.targets[0].id] = (len(value),len(value[0]))
        print("Current Environment:",env)
        return value
    
    def visit_Call(self, node):
        print(type(node))
        if type(node.func) is ast.Attribute:
            if type(node.args[0]) is ast.Name:
                if node.func.attr in self.mult:
                    self.mult.pop(node.func.attr, None)
                    self.mult[node.func.attr] = node.args[0].id
                self.mult[node.func.attr] = [i.id for i in node.args]
            return self.visit(node.args[0])
        
    def visit_List(self, node):
        print(type(node))
        return [self.visit(element) for element in node.elts]
    
    def visit_Expr(self, node):
        print(type(node))
        return self.visit(node.value)
    
    def visit_Attribute(self, node):
        print(type(node), node.attr)
        if node.attr in self.numpyLang:
            return self.visit(node.value)
        
    def visit_BinOp(self, node):
        print(type(node), node.op)
        if node.op in self.binOps:
            return self.visit(node.left)
    
    def visit_Name(self, node):
        print(type(node), node.id)
        if node.id in self.env:
            return self.env[node.id]

    def visit_Num(self, node):
        print(type(node), node.n)
        return node.n
        
    def visit_Return(self, node):
        print(type(node))
        return self.visit(node.value)
    
#Decorator for checking if a module is valid within the syntax
def array_Size():
    def size(f):
        print("Checking for the array sizes of each np.array in the function...")
        print("Checking nodes... \n")
        tree = ast.parse(inspect.getsource(f))
        arraysInEnv, multi = size_Of_Array().visit(tree)
        print("\n")
        # Assuming only 2D Arrays
        print("The Arrays in the function", f.__name__ + "() are of sizes: ")
        for i in arraysInEnv:
            if isinstance(i,list) or isinstance(arraysInEnv[i],tuple):
                print(i + " is a " + str(arraysInEnv[i][0]) + " by " + str(arraysInEnv[i][1]) + " array.")
        print("\n")
        
        if 'dot' in multi:
            beingMultiplied = multi['dot']
        
        mapping = {}
        for array in beingMultiplied:
            mapping[array] = arraysInEnv[array]
        
        print("Arrays Being Multiplied", beingMultiplied)
        
        print("Dimensions of resulting from the matrix multiplication for the two np arrays,", beingMultiplied, "is a:", mapping[beingMultiplied[0]][0],"by",mapping[beingMultiplied[1]][1], "array" )    
        print("\n")
        print("End of Check...")
        print("======================================================================= \n")
    return size

# Decorator to run both the checking for valid language and array sizes... Will only run the Array size decorator only if the program is valid
def valid_And_Size():
    def valid_And_Array_Size(f):
        f_1 = is_valid(dump = True, showRes = True)(f)
        if f_1:
            f_2 = array_Size()(f)
    return valid_And_Array_Size 


# Only works when you np.dot once in the function.
# Also does not account for variables within an array
@valid_And_Size()
def matrix_Size():
    import numpy as np
    x = np.array([[5,2,9],[1,4,7]])
    y = np.array([[99,3],[3,4],[7,3]])
    z = np.array([[9,32],[344,42],[73,32]])
    np.dot(x,z)
    pass


@valid_And_Size()
def matrix_Size2():
    import numpy as np
    a = 2
    x = np.array([[a,2,9],[1,4,7]])
    y = np.array([[99,3],[3,4],[7,3]])
    z = np.array([[9,32,3],[344,42,2],[73,32,2]])
    return np.dot(x,y)


Testing for valid syntax...
Checking nodes... 

<class '_ast.Module'>
<class '_ast.FunctionDef'> : matrix_Size()
<class '_ast.Import'> ['np']
<class '_ast.Assign'>
<class '_ast.Call'>
<class '_ast.Attribute'> array
<class '_ast.Name'> np
<class '_ast.List'>
<class '_ast.List'>
<class '_ast.Num'> 5
<class '_ast.Num'> 2
<class '_ast.Num'> 9
<class '_ast.List'>
<class '_ast.Num'> 1
<class '_ast.Num'> 4
<class '_ast.Num'> 7
<class '_ast.Assign'>
<class '_ast.Call'>
<class '_ast.Attribute'> array
<class '_ast.Name'> np
<class '_ast.List'>
<class '_ast.List'>
<class '_ast.Num'> 99
<class '_ast.Num'> 3
<class '_ast.List'>
<class '_ast.Num'> 3
<class '_ast.Num'> 4
<class '_ast.List'>
<class '_ast.Num'> 7
<class '_ast.Num'> 3
<class '_ast.Assign'>
<class '_ast.Call'>
<class '_ast.Attribute'> array
<class '_ast.Name'> np
<class '_ast.List'>
<class '_ast.List'>
<class '_ast.Num'> 9
<class '_ast.Num'> 32
<class '_ast.List'>
<class '_ast.Num'> 344
<class '_ast.Num'> 42
<class '_ast.List'>
<class '_

# 3(b)
This Next Part Calculates The Memory Of A Matrix For After A NP.Dot Call 

In [78]:
import sys

# def matrix_Size3():
#     import numpy as np
#     x = np.array([[9,32],[344,42],[73,32]])
#     y = np.array([[5,2,9],[1,4,7]])
#     return np.matmul(x,y)

# def matrix_Size4():
#     import numpy as np
#     x = np.array([[9,32],[344,42],[73,32]])
#     y = np.array([[5,2,9],[1,4,7]])
#     return np.dot(x,y)

# f_new_ast = ast.parse(inspect.getsource(matrix_Size3))
# print(ast.dump(f_new_ast))
# print('\n')
# f_new_ast2 = ast.parse(inspect.getsource(matrix_Size4))
# print(ast.dump(f_new_ast2))
class matMul_To_Dot(ast.NodeTransformer):

    def visit_Call(self, node):
        if type(node.func) is ast.Attribute:
            if node.func.attr is 'matmul':
                if type(node.func.value) is ast.Name:
                    if node.func.value.id is 'np':
                        attr = self.visit(node.func)
                        a =\
                        ast.copy_location(
                            ast.Call(
                                func = ast.Attribute(
                                    value = ast.Name(id='np', ctx = ast.Load()),
                                    attr = 'dot',
                                    ctx = ast.Load()
                                ),
                                args = node.args,
                                keywords = node.keywords,
                            ),
                            node
                        )
                        return a
        return node
                    
        
def replace_Mat_Mul():
    import numpy as np
    x = np.array([[9,32],[344,42],[73,32]])
    y = np.array([[5,2,9],[1,4,7]])
    return np.matmul(x,y)

f_new_ast = matMul_To_Dot().visit(ast.parse(inspect.getsource(replace_Mat_Mul)))
print(ast.dump(f_new_ast))
print()
print(astor.to_source(f_new_ast))



def valid_And_Size():
    def valid_And_Array_Size(f):
        f_1 = is_valid(dump = True, showRes = True)(f)
        if f_1:
            f_2 = array_Size()(f)
    return valid_And_Array_Size 




def matrix_Size2():
    import numpy as np
    a = 2
    x = np.array([[a,2,9],[1,4,7]])
    y = np.array([[99,3],[3,4],[7,3]])
    z = np.array([[9,32,3],[344,42,2],[73,32,2]])
    return np.matmul(x,y)

Module(body=[FunctionDef(name='replace_Mat_Mul', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Import(names=[alias(name='numpy', asname='np')]), Assign(targets=[Name(id='x', ctx=Store())], value=Call(func=Attribute(value=Name(id='np', ctx=Load()), attr='array', ctx=Load()), args=[List(elts=[List(elts=[Num(n=9), Num(n=32)], ctx=Load()), List(elts=[Num(n=344), Num(n=42)], ctx=Load()), List(elts=[Num(n=73), Num(n=32)], ctx=Load())], ctx=Load())], keywords=[])), Assign(targets=[Name(id='y', ctx=Store())], value=Call(func=Attribute(value=Name(id='np', ctx=Load()), attr='array', ctx=Load()), args=[List(elts=[List(elts=[Num(n=5), Num(n=2), Num(n=9)], ctx=Load()), List(elts=[Num(n=1), Num(n=4), Num(n=7)], ctx=Load())], ctx=Load())], keywords=[])), Return(value=Call(func=Attribute(value=Name(id='np', ctx=Load()), attr='dot', ctx=Load()), args=[Name(id='x', ctx=Load()), Name(id='y', ctx=Load())], keywords=[]))], decorator_list=[], returns=Non